In [1]:
!pip install git+https://github.com/Evan-Kim2028/subgraph-query-portal.git

  Cloning https://github.com/Evan-Kim2028/subgraph-query-portal.git to /tmp/pip-req-build-loq2yljl
  Running command git clone --filter=blob:none --quiet https://github.com/Evan-Kim2028/subgraph-query-portal.git /tmp/pip-req-build-loq2yljl
  Resolved https://github.com/Evan-Kim2028/subgraph-query-portal.git to commit 7993c20c8a18d0a29a86ac23c7d0f6ff13fcec3f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached python_dotenv-1.0.0-py3-none-any.whl (19 kB)
  Created wheel for queryportal: filename=queryportal-0.0.post1.dev185+g7993c20-py3-none-any.whl size=15133 sha256=435873e4433f0345f95cc96e12c19172a72de381d000c8a647a385cbd361b8fd
  Stored in directory: /tmp/pip-ephem-wheel-cache-i3joux3h/wheels/68/ed/37/5eb5a8cac3e8bd8308342b19c5d94d74d42b0e161e7158c447
Successfully built queryportal
  Attempting uninstall: queryportal
    Found existing installati

In [2]:
from queryportal.subgraphinterface import SubgraphInterface

from datetime import datetime, timedelta


import polars as pl
pl.Config.set_fmt_str_lengths(200)

polars.config.Config

In [5]:
# define subgraph endpoint. This one is the Univ3 Ethereum endpoint maintained by Messari
endpoint = 'https://api.thegraph.com/subgraphs/name/messari/uniswap-v3-ethereum'

# instantiate dex class. Dex stores dex-related query functions
my_dex = SubgraphInterface(endpoint)

First query is to get PEPE pools on Univ3

Tips:
- Find valid query entities and columns by inspecting the endpoint url....

In [6]:
pool_filter = {
    'pool_name_contains': 'Pepe',
    'timestamp_gte': int((datetime(2023, 4, 12).timestamp())),
    'timestamp_lte': int(datetime(2023, 4, 19).timestamp())
}

pool_cols = [
    'timestamp',
    'hash',
    # comes from Pool entity
    'pool_name', 
    'pool_id',
    'pool_totalValueLockedUSD',
    # comes from Token entity
    'tokenIn_symbol', 
    'tokenOut_symbol', 
    'tokenIn_id',
    'tokenOut_id',
    ]

In [8]:
df = my_dex.query_entity(
    entity='swaps',
    name='uniswap-v3-ethereum',
    query_paths=pool_cols,
    query_size=1000,
    filter_dict=pool_filter
    )

Querying subgraph endpoint: uniswap-v3-ethereum
Shape: (1000, 9)
Schema: {'timestamp': Int64, 'hash': Utf8, 'tokenIn_symbol': Utf8, 'tokenIn_id': Utf8, 'tokenOut_symbol': Utf8, 'tokenOut_id': Utf8, 'pool_name': Utf8, 'pool_id': Utf8, 'pool_totalValueLockedUSD': Float64}
Function 'query_entity' executed in 7.1500s


In [9]:
unique_pool_df = df[['pool_name', 'pool_id', 'pool_totalValueLockedUSD']].unique().sort(by='pool_totalValueLockedUSD', descending=True)

In [10]:
unique_pool_df = unique_pool_df.to_pandas()

In [11]:
unique_pool_df

,pool_name,pool_id,pool_totalValueLockedUSD
0,Uniswap V3 McPepes/Wrapped Ether 1%,0xdf19389d0b2bee33230d73f07c8e0e2c4e78fb59,4.286022e+06
1,Uniswap V3 Pepe/Wrapped Ether 1%,0xf239009a101b6b930a527deaab6961b6e7dec8a6,2.686652e+06
2,Uniswap V3 Pepe/USD Coin 1%,0xcee31c846cbf003f4ceb5bbd234cba03c6e940c7,5.422068e+04
3,Uniswap V3 Pepe/McPepes 1%,0x5e63add89fad07394c866b5ec650d062be15ef73,5.250808e+04
4,Uniswap V3 Wojak Coin/Pepe 1%,0x0b4cb9d4fb1cee079392bc7cf6417a669f4c2dbd,3.448001e+02


In [ ]:
# it appears that the main pool liquidity for PEPE is this pool_id 0xf239009a101b6b930a527deaab6961b6e7dec8a6

In [46]:
swap_filter = {
    'timestamp_gte': int((datetime(2023, 4, 19) - timedelta(days=7)).timestamp()),
    'timestamp_lte': int(datetime(2023, 4, 19).timestamp()),
    'pool_id': '0xf239009a101b6b930a527deaab6961b6e7dec8a6'
}

swap_cols = [
    'account_id',
    'tokenIn_symbol', 
    'tokenOut_symbol', 
    'amountOutUSD', 
    'amountInUSD',
    'amountOut',
    'amountIn'
    ]

swap_df = my_dex.query_swaps(query_paths=swap_cols, filter_dict=swap_filter, query_size=100000, saved_file_name='pepe_swaps_4.12.23_to_4.19.23', add_endpoint_col=False)


{'pool_': {'id': '0xf239009a101b6b930a527deaab6961b6e7dec8a6'}}
account
tokenIn
tokenOut
Shape: (28112, 9)
Column Names: ['amountOutUSD', 'amountInUSD', 'amountOut', 'amountIn', 'id', 'timestamp', 'account_id', 'tokenIn_symbol', 'tokenOut_symbol']
Data Types: [Float64, Float64, Float64, Float64, Utf8, Utf8, Utf8, Utf8, Utf8]
Data: 
shape: (5, 9)
┌──────────────┬─────────────┬───────────┬───────────┬───┬────────────┬───────────────────────────────────┬────────────────┬─────────────────┐
│ amountOutUSD ┆ amountInUSD ┆ amountOut ┆ amountIn  ┆ … ┆ timestamp  ┆ account_id                        ┆ tokenIn_symbol ┆ tokenOut_symbol │
│ ---          ┆ ---         ┆ ---       ┆ ---       ┆   ┆ ---        ┆ ---                               ┆ ---            ┆ ---             │
│ f64          ┆ f64         ┆ f64       ┆ f64       ┆   ┆ str        ┆ str                               ┆ str            ┆ str             │
╞══════════════╪═════════════╪═══════════╪═══════════╪═══╪════════════╪═════════